## LangChain Overview

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_anthropic import ChatAnthropic
llm_claude = ChatAnthropic(model="claude-3-opus-20240229")

from langchain_openai import ChatOpenAI
llm_gpt = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
llm_claude.invoke("What is LangChain?").content

In [ ]:
llm_gpt.invoke("What is LangChain?").content

In [ ]:
system_message = """You explain things to the people like they are five years old."""
user_prompt = f"What is LangChain?"

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
import textwrap

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content=user_prompt)
]

In [ ]:
response = llm_claude.invoke(messages)
answer = textwrap.fill(response.content, width=100)
print(answer)

## Chains

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
You are a helpful assistant who explain blockchain and metaverse concepts. Given an input topic {topic},
explain it to the user like they are five years old."""

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

In [ ]:
# assemble the chain using pipe "|" operator
chain = prompt | llm_claude

# invoke the chain
response = chain.invoke(topic="What is Metaverse?")
print(response.content)

## Document Loaders

In [ ]:
pip install --upgrade --quiet youtube-transcript-api

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

loader = YoutubeLoader.from_youtube_url("https://youtube.com/shorts/7eT12YMA8oE?si=E3LBCPNDjvn1IYkA", 
                                        add_video_info=False)

In [ ]:
docs = loader.load()

In [ ]:
docs